In [ ]:
import sys
sys.path
!pip install twitter
!pip install python-geohash
!pip install timehash
!pip install polyglot
!pip install scikit-learn
!pip install stop-words
!pip install pandas
!pip install pandas-ply


In [3]:
western_world = """Europe (at least the European Union member states, EFTA countries, European microstates);
in the Americas (e.g. Argentina, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Mexico, 
United States of America, Uruguay, Venezuela), and in Oceania (Australia and New Zealand)"""

western_countries = []

In [4]:
import json
import re

#shameless copy paste from json/decoder.py
FLAGS = re.VERBOSE | re.MULTILINE | re.DOTALL
WHITESPACE = re.compile(r'[ \t\n\r]*', FLAGS)

class ConcatJSONDecoder(json.JSONDecoder):
    def decode(self, s, _w=WHITESPACE.match):
        s_len = len(s)

        objs = []
        end = 0
        while end != s_len:
            obj, end = self.raw_decode(s, idx=_w(s, end).end())
            end = _w(s, end).end()
            objs.append(obj)
        return objs

In [5]:
import geohash, timehash, os, json, string, sklearn,unicodedata, pandas as pd

In [6]:
path = os.path.normpath(os.path.join(os.path.normpath(os.path.expanduser("~")),"projects","LC3-Creations"))

In [7]:
path

'/Users/linwood/projects/LC3-Creations'

In [8]:
oauth = json.load(open(os.path.join(path,"apikeys.txt")), cls=ConcatJSONDecoder)

In [9]:
#**********************************************************************
# Passing in oauth values; you obviscate by loading your locally stored json
#**********************************************************************

# Import the necessary package to process data in JSON format
try:
    import json
except ImportError:
    import simplejson as json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

# Variables that contains the user credentials to access Twitter API 
# If you have not followed the obfuscate process above, this will be empty and error out
ACCESS_TOKEN = oauth[0]['twitter']['accesstoken']
ACCESS_SECRET = oauth[0]['twitter']['accesssecret']
CONSUMER_KEY = oauth[0]['twitter']['consumerkey']
CONSUMER_SECRET = oauth[0]['twitter']['consumersecret']

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

In [ ]:
#**********************************************************************
# Code to pull tweets from twitter stream

# I pulled this code from http://socialmedia-class.org/twittertutorial.html.  
# I used the locations filter, but you can alter to get a sample or pull 
# specific keywords.  Use the link above.  The only line you would alter is:
# iterator = twitter_stream.statuses.filter()
# Want to know what arguments to pass in? visit https://dev.twitter.com/streaming/reference/post/statuses/filter
#**********************************************************************


# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)

# Filter the public data following through Twitter; the format is "long,lat , long,lat" with southwest corner first
iterator = twitter_stream.statuses.filter(locations = '-44.888519, -23.37085, -40.953869,-20.76347')

# Print each tweet in the stream to the screen 
# Here we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer. 

with open('./output/<brazilSample.txt'>, 'w+') as outfile:
    for tweet in iterator:
        try:
            if tweet['user']['geo_enabled']==True:
                json.dump(tweet, outfile)
        except:
            pass
            
        
        # Twitter Python Tool wraps the data returned by Twitter 
        # as a TwitterDictResponse object.
        # We convert it back to the JSON format to print/score
        #print json.dumps(tweet)  

        # The command below will do pretty printing for JSON data, try it out
        # print json.dumps(tweet, indent=4)
        
    outfile.closed  

In [10]:
brazil = json.load(open(os.path.join(path,"notebooks","twittertest","output","brazilSample.txt")), cls=ConcatJSONDecoder)

In [11]:
len(brazil)

3454

In [12]:
brazil[0].keys()

dict_keys(['coordinates', 'favorited', 'source', 'in_reply_to_status_id_str', 'created_at', 'favorite_count', 'is_quote_status', 'text', 'place', 'filter_level', 'in_reply_to_screen_name', 'contributors', 'geo', 'in_reply_to_status_id', 'id_str', 'id', 'truncated', 'retweeted', 'retweet_count', 'timestamp_ms', 'in_reply_to_user_id', 'user', 'in_reply_to_user_id_str', 'entities', 'lang'])

In [15]:
braz_frame = pd.DataFrame(brazil)
braz_frame = braz_frame[["id","created_at","place","text","lang","geo","entities","timestamp_ms","user","coordinates"]] 

In [18]:
braz_frame['timestamp_ms'] = braz_frame['timestamp_ms'].to_timestamp

In [30]:
import geohash
print (geohash.encode(-22.969051, -43.187769))
print (geohash.encode(-22.969486, -43.179830))
print (geohash.encode(-22.966404, -43.174465))
print (geohash.encode(-22.964547, -43.170131))
print (geohash.encode(-22.960793, -43.165324))

75cm34xes493
75cm36t3j13n
75cm3e16fpcx
75cm3ekmz41j
75cm3spgxjcw


In [ ]:


# decode('utf-8') changed from bytes to string
tweets = [unicodedata.normalize('NFKD', i['text']).encode('ascii','ignore').decode('utf-8') for i in brazil]

In [ ]:
len(tweets),type(tweets)
tweets[1]

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
word_tokenize(tweets[5])

In [ ]:
set(word_tokenize(tweets[5])) - set(string.punctuation)-set(stop_words)

In [ ]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

In [ ]:
set(stopwords.words('portuguese')) & set(stop_words)

In [ ]:
stop_words = get_stop_words('portuguese')

# MongoDB practice on Tweets

In [33]:
from pymongo import MongoClient,GEO2D
import pymongo



In [58]:
client = MongoClient('localhost', 27017)
db = client.brazil
collection = db['brazil-tweets']
brazil_tweets = db['brazil-tweets']

# Creating a geospatial index for querying

In [235]:
cut = brazil_tweets.find({"coordinates.coordinates":{"$ne":None}})
for doc in cut:
    docID = doc['_id']
    doc['geometry']= {"type":"Point", "coordinates": [float(doc['coordinates']['coordinates'][0]),
                      float(doc['coordinates']['coordinates'][1])]}
    brazil_tweets.update_one({'_id':docID},{"$set":doc}, upsert=False)
    
brazil_tweets.create_index([("geometry",pymongo.GEOSPHERE)])

In [250]:
boundary = {}
boundary['geometry']={"type":"Polygon","coordinates":[[["-43.793888092041","-23.0763473510742"],["-42.6359710693359","-22.4493675231934"]]]}

In [ ]:
for l in brazil_tweets.find({"geometry":{"$ne":None}}):
    print (l['text'])
    

In [259]:
for l in brazil_tweets.find({"geometry": {"$geoWithin": {"$box": [[-23.006508, -22.946076],
                                                               [-43.254849, -43.171159]]}}}):
    print (l['text'])

3⃣0⃣4⃣❗😍⛴🛳🚢✨🇺🇾🇧🇷🌴☀💥💃👗👠👑🎆🎈💕💖 #Crucero2017 #Miss15 @ MSC Orchestra… https://t.co/URLiMSpgJD
Lagoa ❤️ @ Lagoa Rodrigo De Freitas - Rio De Janeiro - Brasil https://t.co/9j1FV8f1LF
Mais um filho nasceu, nossa verdade musical, uma proposta diferente, pra encantar! É impossível… https://t.co/aAKQzmeedw
Ai, que saudade docês! De você também, @wiledsilveira , mas não achei fofo do quarteto ! @… https://t.co/LAjsh2aLby
Mais um dia de recreação na @riofighters Sábado é dia das crianças #RioMiniFighters… https://t.co/JqbcQ6iKEs
Nada de mais... Simplesmente eu! Minha nova tatuagem em fase inicial feita pelo Eduardo meu anjo… https://t.co/Os3INoPxah
I'm at Amelie in Rio de Janeiro https://t.co/uD78CXnjiL
Sinceridade gata, eh nois que ta linda 🌊🌺💣 @ Praia de Itacoatiara https://t.co/V4yrToNXeT
I'm at Praia de Ipanema in Rio de Janeiro, RJ, RJ https://t.co/HFDIseYs4F
Just posted a photo @ Estátua de Dorival Caymmi https://t.co/BxaJ9S088k
🍻 (@ Maria da Praia in Niterói, RJ) https://t.co/Hr61Z8eW0E
Y no

In [118]:
from bson.son import SON
query = {"coordinates": SON([("$near", [-43.40184053, -22.78785425]), ("$maxDistance", 1000)])}
for doc in db.brazil_tweets.find(query).limit(100):
    print (doc)

In [277]:
for doc in brazil_tweets.find({"geometry": {"$within": {"$center": [[ -43.182651,-22.966658], .01]}}}):
    print (doc['text'],doc['geometry']['coordinates'])

3⃣0⃣4⃣❗😍⛴🛳🚢✨🇺🇾🇧🇷🌴☀💥💃👗👠👑🎆🎈💕💖 #Crucero2017 #Miss15 @ MSC Orchestra… https://t.co/URLiMSpgJD [-43.18994352, -22.97149778]
Mais um filho nasceu, nossa verdade musical, uma proposta diferente, pra encantar! É impossível… https://t.co/aAKQzmeedw [-43.19110816, -22.96417801]


In [288]:
import polyglot
from polyglot.text import Text, Word


word = Word("protesto", language="pt")
for w in word.neighbors:
    print (w)

genocídio
motim
levante
pronunciamento
comício
boicote
retaliação
agradecimento
tumulto
assédio


In [292]:
word.vector

array([ 0.59737962, -0.64523113,  0.33711299, -0.3455652 ,  0.19411458,
       -0.33565947, -2.62641382, -0.22692174,  0.56430954,  0.37213761,
        1.62610054, -0.97020465, -2.01416898, -0.02478787,  0.75397676,
       -1.30008435,  1.35396922,  1.00399423, -0.82601434,  2.21568418,
       -0.91054553, -0.12676898,  1.53243148, -1.0757668 , -0.28473505,
       -0.41047156, -1.6206857 ,  1.80200827, -0.42291695, -1.82027614,
        0.89114261, -1.53802562, -0.85561776,  1.32964885, -0.00562795,
        1.07157397, -0.04918216,  0.05134355,  0.71893853, -1.5087105 ,
        0.03973555, -1.74955964,  0.56620049, -2.9240346 ,  0.2276551 ,
        0.40591595,  0.67361599,  2.29726338,  1.00404775,  3.56751752,
       -0.43566847,  1.67288542,  0.02764815, -2.25402856,  0.66003841,
       -1.61814737, -1.27222264,  0.54359263,  3.7455554 ,  0.06947216,
       -0.77350634,  1.52999365, -1.38145363, -2.08945274, -1.05367637,
       -0.38085824,  1.29245329,  2.81955838, -3.83165956,  1.18